In [1]:
import pandas

In [2]:
dataset = pandas.read_excel("dataset.xlsx")

In [3]:
# From the main dataset, split into training and test dataframes
training_set = dataset.loc[dataset['Tr=traning set\nTe=test set'] == "Tr"]
test_set = dataset.loc[dataset['Tr=traning set\nTe=test set'] == "Te"]

# Trim Wang's (2015) predicted values since we are calculating these ourselves
training_set = training_set.iloc[:,0:4]
test_set = test_set.iloc[:,0:4]

print("Training set has {} items, test set has {} items".format(training_set.iloc[:,0].count(), test_set.iloc[:,0].count()))

Training set has 904 items, test set has 226 items


In [4]:
training_set

,NO,Tr=traning set Te=test set,SMILES structure,experimental logD7.4
1,A-logD7.4-100,Tr,Fc1cc2c(N(C=C(C(O)=O)C2=O)C2CC2)cc1N1CCNCC1,-0.960
2,A-logD7.4-101,Tr,S1(=O)(=O)N(CCN1C)c1cc2c([nH]cc2CCN(C)C)cc1,-0.920
3,A-logD7.4-104,Tr,OC=1C(=O)C=CN(CCCO)C=1C,-0.900
4,A-logD7.4-105,Tr,Fc1cc2c(N(C=C(C(O)=O)C2=O)c2ccccc2C)cc1N1CCNCC1,-0.830
5,A-logD7.4-106,Tr,OC=1C(=O)C=CN(CCC(=O)NCC)C=1C,-0.820
6,A-logD7.4-108,Tr,Fc1cc2c(N(C=C(C(O)=O)C2=O)C2CC2)c(OC)c1N1CC(NC...,-0.790
7,A-logD7.4-109,Tr,S1(=O)(=O)NCC(N1Cc1cc2c([nH]cc2CCN(C)C)cc1)(C)C,-0.780
8,A-logD7.4-110,Tr,OC=1C(=O)C=CN(CCC(OC)=O)C=1C,-0.770
9,A-logD7.4-111,Tr,OC=1C(=O)C=CN(CC(O)(C)C)C=1C,-0.770
10,A-logD7.4-112,Tr,OC=1C(=O)C=CN(C)C=1C,-0.770
